# Part 6: AgentReview Demo - 5단계 Peer Review 시뮬레이션

**SNU AI Psychology Workshop - February 2026**

---

## AgentReview (EMNLP 2024)

실제 **ICLR** (International Conference on Learning Representations) peer review 프로세스를 LLM으로 시뮬레이션하는 프레임워크입니다.

**5단계 파이프라인:**
1. **Reviewer Assessment**: 3명의 리뷰어가 독립적으로 논문 평가
2. **Author-Reviewer Discussion**: 저자가 rebuttal 제출
3. **Reviewer-AC Discussion**: Area Chair가 리뷰어 간 토론 중재
4. **Meta-Review**: AC가 종합 리뷰 작성
5. **Paper Decision**: Accept/Reject 최종 결정

**참고 자료:**
- 📄 [논문](https://arxiv.org/abs/2406.12708)
- 🌐 [웹사이트](https://agentreview.github.io/)
- 💻 [GitHub](https://github.com/Ahren09/AgentReview)
- 💡 [데모](https://huggingface.co/spaces/Ahren09/AgentReview)

**데이터:**
- 이 노트북은 `data/ICLR2022/` 폴더의 샘플 논문 4개를 사용합니다.
- 전체 데이터셋(선택): [Dropbox](https://www.dropbox.com/scl/fi/l17brtbzsy3xwflqd58ja/AgentReview_Paper_Data.zip?rlkey=vldiexmgzi7zycmz7pumgbooc&st=b6g3nkry&dl=0)

---
## Setup

In [ ]:
# Cell 1: 환경 설정
import os
import sys

try:
    from google.colab import drive
    drive.mount('/content/drive/')
    WORKSHOP_DIR = "/content/drive/MyDrive/aiworkshop_Feb2026/"
    os.makedirs(WORKSHOP_DIR, exist_ok=True)
    os.chdir(WORKSHOP_DIR)
    IN_COLAB = True
    print("🌐 Colab 환경")
except ImportError:
    # 로컬: 프로젝트 루트로 이동
    current_dir = os.getcwd()
    if current_dir.endswith('notebooks'):
        os.chdir('..')
    WORKSHOP_DIR = os.getcwd()
    IN_COLAB = False
    print("💻 로컬 환경")

print(f"작업 폴더: {WORKSHOP_DIR}")

In [ ]:
# Cell 2: 패키지 설치
if IN_COLAB:
    print("📦 설치 중...")
    %pip install openai python-dotenv llama-index-readers-file -q
    !git clone https://github.com/ahren09/agentreview.git 2>/dev/null || echo "✓ AgentReview 이미 존재"
    print("✅ 완료!")
else:
    print("✅ 로컬: pyproject.toml 사용")
    import subprocess
    import os
    
    # WORKSHOP_DIR 기준으로 확인
    agentreview_path = os.path.join(WORKSHOP_DIR, "agentreview")
    
    if not os.path.exists(agentreview_path):
        print("📥 AgentReview 클론 중...")
        subprocess.run(["git", "clone", "https://github.com/ahren09/agentreview.git", agentreview_path], capture_output=True)
        print("✅ AgentReview 클론 완료")
    else:
        print("✓ AgentReview 이미 존재")
    
    # llama-index 설치 확인
    try:
        import llama_index
        print("✓ llama-index 설치됨")
    except ImportError:
        print("📥 llama-index 설치 중...")
        subprocess.run(["pip", "install", "llama-index-readers-file"], capture_output=True)
        print("✅ llama-index 설치 완료")

In [ ]:
# Cell 3: API Key
if IN_COLAB:
    try:
        from google.colab import userdata
        OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
        print("✅ Colab Secrets에서 로딩")
    except:
        OPENAI_API_KEY = None
        print("⚠️ Colab Secrets에 OPENAI_API_KEY 추가 필요")
else:
    try:
        from dotenv import load_dotenv
        load_dotenv()
        OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
        if OPENAI_API_KEY:
            print("✅ .env 파일에서 로딩")
        else:
            print("⚠️ .env 파일에 OPENAI_API_KEY 추가 필요")
    except:
        OPENAI_API_KEY = None

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY or ''
print(f"\n🔑 OpenAI: {'설정됨' if OPENAI_API_KEY else '없음'}")

---
## 데이터 확인

In [ ]:
# Cell 4: 샘플 데이터 확인
import os
import json

data_dir = "data/ICLR2022"

if os.path.exists(data_dir):
    print("✅ ICLR2022 샘플 데이터 발견!")
    
    # 논문 ID 확인
    with open(f"{data_dir}/id2decision.json") as f:
        id2decision = json.load(f)
    
    print(f"\n📄 포함된 논문 ({len(id2decision)}개):")
    for pid, decision in id2decision.items():
        print(f"   - Paper {pid}: {decision}")
    
    USE_AGENTREVIEW = True
else:
    print(f"❌ 샘플 데이터를 찾을 수 없습니다: {os.path.abspath(data_dir)}")
    print(f"   현재 작업 폴더: {os.getcwd()}")
    print(f"\n해결 방법:")
    print(f"   1. 프로젝트 루트 폴더에서 노트북 실행")
    print(f"   2. 또는 Cell 1을 먼저 실행하여 작업 폴더 설정")
    USE_AGENTREVIEW = False

---
## 리뷰어 페르소나 설정

In [ ]:
# Cell 5: 리뷰어 설정

# 리뷰어 페르소나 속성 (조합 가능!)
# 
# 1. Intention (의도):
#    - benign: 호의적, 높은 점수 선호
#    - malicious: 악의적, 낮은 점수 선호
#    - BASELINE: 중립
#
# 2. Knowledgeability (전문성):
#    - knowledgeable: 전문가 수준
#    - unknowledgeable: 비전문가
#    - BASELINE: 보통
#
# 3. Commitment (책임감):
#    - responsible: 성실한 리뷰
#    - irresponsible: 불성실한 리뷰
#    - BASELINE: 보통
#
# 4. Fame (저자 인지도):
#    - authors_are_famous: 유명 저자로 인식
#    - authors_are_unfamous: 무명 저자로 인식
#
# 조합 예시:
#    - "knowledgeable_responsible": 전문가이면서 성실
#    - "malicious_unknowledgeable": 악의적이면서 비전문가
#    - "benign_irresponsible": 호의적이지만 불성실

# 기본 설정: 모든 리뷰어가 일반적 (BASELINE)
baseline_setting = {
    "reviewer": ["BASELINE", "BASELINE", "BASELINE"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

# 실험 1: 악의적 리뷰어 1명
malicious_setting = {
    "reviewer": ["malicious", "BASELINE", "BASELINE"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

# 실험 2: 호의적 리뷰어 1명
benign_setting = {
    "reviewer": ["benign", "BASELINE", "BASELINE"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

# 실험 3: 전문가 vs 비전문가 혼합
expert_mix_setting = {
    "reviewer": ["knowledgeable_responsible", "unknowledgeable", "BASELINE"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

# 실험 4: 최악의 시나리오 (악의적 + 비전문가 + 불성실)
worst_case_setting = {
    "reviewer": ["malicious_unknowledgeable_irresponsible", "BASELINE", "BASELINE"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

# 실험 5: 유명 저자 효과 테스트
fame_bias_setting = {
    "reviewer": ["authors_are_famous", "authors_are_unfamous", "BASELINE"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

print("✅ 리뷰어 페르소나 정의 완료")
print("\n사용 가능한 설정:")
print("  1. baseline_setting: 모두 중립")
print("  2. malicious_setting: 악의적 리뷰어 1명")
print("  3. benign_setting: 호의적 리뷰어 1명")
print("  4. expert_mix_setting: 전문가 vs 비전문가")
print("  5. worst_case_setting: 최악의 리뷰어")
print("  6. fame_bias_setting: 저자 유명도 편향 테스트")
print("\n💡 Cell 7에서 원하는 설정을 선택하세요!")

---
## AgentReview 실행

In [ ]:
# Cell 6: AgentReview 모듈 로딩
if USE_AGENTREVIEW and OPENAI_API_KEY:
    import sys
    sys.path.insert(0, os.path.abspath("agentreview"))
    
    try:
        from agentreview.environments import PaperReview
        from agentreview.paper_review_arena import PaperReviewArena
        from agentreview.paper_review_settings import get_experiment_settings
        from agentreview.utility.experiment_utils import initialize_players
        from agentreview.utility.utils import get_paper_decision_mapping
        
        print("✅ AgentReview 모듈 로딩 완료")
        READY = True
    except Exception as e:
        print(f"❌ AgentReview 모듈 로딩 실패")
        print(f"   오류: {e}")
        print(f"\n해결 방법:")
        print(f"   - Cell 2를 먼저 실행하여 AgentReview 클론")
        READY = False
elif not USE_AGENTREVIEW:
    print("⚠️ 샘플 데이터가 없어 실행할 수 없습니다.")
    print("   Cell 4를 확인하세요.")
    READY = False
elif not OPENAI_API_KEY:
    print("⚠️ OpenAI API Key가 없어 실행할 수 없습니다.")
    print("   .env 파일에 OPENAI_API_KEY를 추가하세요.")
    READY = False

In [ ]:
# Cell 7: 리뷰 생성하기
#
# ⚠️ 중요: AgentReview 실행은 시간이 오래 걸리고 OpenAI API 비용이 발생합니다.
#
# 📌 현재 포함된 샘플 결과는 불완전합니다 (API quota 오류로 중단됨).
#    완전한 5단계 리뷰를 보려면 아래 코드의 주석을 해제하고 실행하세요.
#    (OpenAI API 키와 충분한 quota가 필요합니다)

# if READY:
#     import sys
#     
#     # 🔧 여기서 리뷰어 설정 변경 가능!
#     selected_setting = baseline_setting  # <- 이 부분을 바꿔보세요!
#     
#     # AgentReview의 parse_args() 사용
#     sys.argv = [
#         'notebook',
#         '--conference', 'ICLR2022',
#         '--model_name', 'gpt-4o',
#         '--data_dir', './data',
#         '--experiment_name', 'BASELINE'
#     ]
#     
#     from agentreview.arguments import parse_args
#     args = parse_args()
#     args.task = "paper_review"
#     
#     paper_id2decision, paper_decision2ids = get_paper_decision_mapping(
#         args.data_dir, args.conference
#     )
#     
#     sample_ids = [284, 216]
#     
#     for paper_id in sample_ids:
#         paper_decision = paper_id2decision[paper_id]
#         experiment_setting = get_experiment_settings(
#             paper_id=paper_id,
#             paper_decision=paper_decision,
#             setting=selected_setting
#         )
#         
#         players = initialize_players(
#             experiment_setting=experiment_setting, 
#             args=args
#         )
#         
#         env = PaperReview(
#             player_names=[p.name for p in players],
#             paper_decision=paper_decision,
#             paper_id=paper_id,
#             args=args,
#             experiment_setting=experiment_setting
#         )
#         
#         arena = PaperReviewArena(
#             players=players, 
#             environment=env, 
#             args=args
#         )
#         
#         arena.launch_cli(interactive=False)
#     
#     print("\n✅ 리뷰 생성 완료! Cell 9에서 display_review_phases()를 실행하여 결과를 확인하세요.")

# 현재 상태 확인
import os
from pathlib import Path

print("📊 현재 저장된 리뷰 결과:\n")

review_dir = Path("outputs/paper_review/ICLR2022/gpt-4o/BASELINE")
if review_dir.exists():
    papers = sorted([d.name for d in review_dir.iterdir() if d.is_dir()])
    for paper_id in papers:
        json_file = review_dir / paper_id / f"{paper_id}.json"
        if json_file.exists():
            print(f"⚠️  Paper {paper_id}: 불완전 (API quota 오류)")
    
    print(f"\n💡 완전한 리뷰를 생성하려면 위 코드의 주석을 해제하고 실행하세요.")
else:
    print("⚠️  샘플 결과가 없습니다.")
    print("   위 코드의 주석을 해제하고 실행하여 새로운 리뷰를 생성하세요.")

---
## 결과 확인

In [ ]:
# Cell 8: 빠른 시작 가이드

print("🚀 AgentReview 데모 사용법\n")
print("="*60)
print()
print("⚠️  중요: 현재 샘플 데이터는 불완전합니다 (API quota 오류)")
print()
print("새로운 완전한 리뷰를 생성하려면:")
print()
print("1️⃣ OpenAI API 키 확인")
print("   - Cell 3에서 API 키가 설정되었는지 확인")
print()
print("2️⃣ 리뷰어 설정 선택 (선택사항)")
print("   - Cell 5에서 다양한 리뷰어 페르소나 확인")
print("   - Cell 7에서 selected_setting 변경")
print()
print("3️⃣ 리뷰 생성 실행")
print("   - Cell 7의 주석 해제")
print("   - 실행 (약 5-10분 소요 + API 비용)")
print()
print("4️⃣ 결과 확인")
print("   - Cell 9: display_review_phases(284)")
print("   - Cell 9: display_review_phases(216)")
print()
print("="*60)

In [ ]:
# Cell 9: 리뷰 과정 단계별 확인
import json
import re
from pathlib import Path

def load_review_history(paper_id, conference, model_name, experiment_name='BASELINE'):
    """AgentReview가 저장한 리뷰 히스토리 로드"""
    # AgentReview의 실제 저장 경로 (수정됨)
    review_path = Path(f"outputs/paper_review/{conference}/{model_name}/{experiment_name}/{paper_id}/{paper_id}.json")
    
    if not review_path.exists():
        return None
    
    with open(review_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    return data

def extract_score(content):
    """리뷰 content에서 점수 추출"""
    # "Overall Score: 6" 또는 "Rating: 6/10" 패턴
    patterns = [
        r'Overall Score:\s*(\d+)',
        r'Rating:\s*(\d+)',
        r'Score:\s*(\d+)',
    ]
    
    for pattern in patterns:
        match = re.search(pattern, content, re.IGNORECASE)
        if match:
            return int(match.group(1))
    return None

def display_review_phases(paper_id, conference='ICLR2022', model_name='gpt-4o', experiment_name='BASELINE'):
    """5단계 리뷰 과정을 단계별로 출력"""
    
    data = load_review_history(paper_id, conference, model_name, experiment_name)
    
    if not data:
        print(f"❌ Paper {paper_id}의 리뷰 결과를 찾을 수 없습니다.")
        print(f"   먼저 Cell 7의 주석을 해제하고 실행하여 리뷰를 생성하세요.")
        return
    
    messages = data['messages']
    experiment_setting = data['experiment_setting']
    
    # API 오류로 불완전한 데이터 확인
    if len(messages) < 3:
        print(f"\n⚠️  Paper {paper_id}의 리뷰 데이터가 불완전합니다.")
        print(f"   API quota 오류로 리뷰 생성이 중단되었습니다.")
        print(f"\n   완전한 리뷰를 보려면:")
        print(f"   1. OpenAI API 키가 유효한지 확인")
        print(f"   2. Cell 7의 주석을 해제하고 실행")
        print(f"\n   저장된 메시지 수: {len(messages)}개")
        return
    
    print(f"\n{'='*80}")
    print(f"📄 Paper {paper_id} - 5단계 Peer Review 과정")
    print(f"{'='*80}\n")
    
    # 리뷰어 설정 출력
    reviewer_types = experiment_setting['players']['Reviewer']
    print("🎭 리뷰어 설정:")
    for i, r_type in enumerate(reviewer_types, 1):
        # BASELINE 설정은 모든 속성이 null
        if all(v is None for v in r_type.values()):
            persona = "BASELINE (중립)"
        else:
            persona = str(r_type)
        print(f"   Reviewer {i}: {persona}")
    print()
    
    # Phase 1: 초기 리뷰
    print(f"\n{'─'*80}")
    print("📝 Phase 1: Reviewer Assessment (초기 리뷰)")
    print(f"{'─'*80}\n")
    
    initial_scores = {}
    for msg in messages:
        if msg['agent_name'].startswith('Reviewer') and 'Phase 1' in msg.get('msg_type', ''):
            reviewer = msg['agent_name']
            score = extract_score(msg['content'])
            initial_scores[reviewer] = score
            
            print(f"\n[{reviewer}]")
            if score:
                print(f"점수: {score}/10")
            # 리뷰 요약 (첫 300자)
            content_preview = msg['content'][:300].replace('\n', ' ')
            print(f"리뷰: {content_preview}...")
            print()
    
    # Phase 2: Author Rebuttal
    print(f"\n{'─'*80}")
    print("✍️ Phase 2: Author-Reviewer Discussion (Rebuttal)")
    print(f"{'─'*80}\n")
    
    for msg in messages:
        if msg['agent_name'] == 'Author':
            print(f"\n[Author → {msg['visible_to']}]")
            content_preview = msg['content'][:300].replace('\n', ' ')
            print(f"{content_preview}...")
            print()
    
    # Phase 3: AC Discussion (업데이트된 리뷰)
    print(f"\n{'─'*80}")
    print("💬 Phase 3: Reviewer-AC Discussion (Rebuttal 후 업데이트)")
    print(f"{'─'*80}\n")
    
    updated_scores = {}
    for msg in messages:
        if msg['agent_name'].startswith('Reviewer') and 'Phase 3' in msg.get('msg_type', ''):
            reviewer = msg['agent_name']
            score = extract_score(msg['content'])
            updated_scores[reviewer] = score
            
            print(f"\n[{reviewer}]")
            if score:
                initial = initial_scores.get(reviewer)
                if initial:
                    change = score - initial
                    arrow = "↑" if change > 0 else "↓" if change < 0 else "→"
                    print(f"점수: {initial} {arrow} {score} ({change:+d})")
                else:
                    print(f"점수: {score}/10")
            content_preview = msg['content'][:200].replace('\n', ' ')
            print(f"업데이트: {content_preview}...")
            print()
    
    # Phase 4: AC Meta-review
    print(f"\n{'─'*80}")
    print("🎯 Phase 4: AC Meta-Review")
    print(f"{'─'*80}\n")
    
    for msg in messages:
        if msg['agent_name'] == 'AC' and 'meta' in msg['content'].lower():
            content_preview = msg['content'][:400].replace('\n', ' ')
            print(f"[AC]")
            print(f"{content_preview}...")
            print()
    
    # 점수 변화 요약
    print(f"\n{'─'*80}")
    print("📊 점수 변화 요약")
    print(f"{'─'*80}\n")
    
    for reviewer in sorted(initial_scores.keys()):
        initial = initial_scores.get(reviewer)
        updated = updated_scores.get(reviewer, initial)
        if initial and updated:
            change = updated - initial
            arrow = "↑" if change > 0 else "↓" if change < 0 else "→"
            print(f"{reviewer}: {initial} {arrow} {updated} ({change:+d})")
    
    print(f"\n{'='*80}\n")

# 사용 예시
print("💡 사용법:")
print("   display_review_phases(284)  # Paper 284의 리뷰 과정 확인")
print("   display_review_phases(216)  # Paper 216의 리뷰 과정 확인")
print("\n⚠️  현재 샘플 데이터는 불완전합니다. Cell 7을 실행하여 완전한 리뷰를 생성하세요.")

In [ ]:
# DIY Cell: 리뷰어 페르소나 탐색하기

# 💡 이 데모의 핵심: 리뷰어 성향이 결과에 미치는 영향 비교하기!
#
# 📌 현재 샘플 데이터는 불완전하므로, 먼저 Cell 7을 실행하여 완전한 리뷰를 생성하세요.

# 🎭 실험 아이디어:
#
# 1️⃣ 기본 실험: BASELINE vs 다른 설정 비교
#    - Cell 7: selected_setting = baseline_setting, experiment_name = 'BASELINE'
#    - 실행 후 결과 확인
#    - Cell 7: selected_setting = malicious_setting, experiment_name = 'malicious'
#    - 실행 후 결과 비교
#
# 2️⃣ 리뷰 결과 확인:
#    display_review_phases(284, experiment_name='BASELINE')
#    display_review_phases(284, experiment_name='malicious')
#
# 3️⃣ 비교할 점:
#    - 초기 점수가 다른가요?
#    - Rebuttal 후 점수 변화가 다른가요?
#    - 어떤 리뷰어가 더 설득력 있나요?

# 🔍 다양한 리뷰어 조합 시도해보기:
# 
# Cell 5에서 정의된 설정들:
# - baseline_setting: 모두 중립
# - malicious_setting: 악의적 리뷰어 1명
# - benign_setting: 호의적 리뷰어 1명
# - expert_mix_setting: 전문가 vs 비전문가
# - worst_case_setting: 최악의 리뷰어
# - fame_bias_setting: 저자 유명도 편향

print("💡 리뷰어 페르소나 탐색 가이드")
print()
print("1. Cell 7에서 selected_setting 변경")
print("2. experiment_name도 변경 (예: 'malicious', 'benign')")
print("3. 주석 해제 후 실행")
print("4. display_review_phases(284, experiment_name='설정이름')으로 확인")
print()
print("⚠️  먼저 완전한 리뷰를 생성해야 합니다 (Cell 7 실행 필요)")

---
## 설정 비교 실험

---
## DIY: 설정 비교 실험

In [ ]:
# DIY Cell 9: 나만의 리뷰어 조합 만들기

# 조합 가능한 속성:
# - Intention: benign (호의적), malicious (악의적), BASELINE (중립)
# - Knowledge: knowledgeable (전문가), unknowledgeable (비전문가), BASELINE (보통)
# - Commitment: responsible (성실), irresponsible (불성실), BASELINE (보통)
# - Fame: authors_are_famous, authors_are_unfamous
#
# 언더스코어(_)로 연결하여 조합:
# 예: "knowledgeable_benign", "malicious_unknowledgeable_irresponsible"

# 실험 아이디어 1: 전문가들만
expert_only_setting = {
    "reviewer": ["knowledgeable_responsible", "knowledgeable_benign", "knowledgeable"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

# 실험 아이디어 2: 다양한 성향 혼합
diverse_setting = {
    "reviewer": ["knowledgeable_malicious", "benign_irresponsible", "unknowledgeable_responsible"],
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

# 실험 아이디어 3: 당신만의 조합 (아래 수정!)
my_custom_setting = {
    "reviewer": ["BASELINE", "BASELINE", "BASELINE"],  # <- 여기를 바꿔보세요!
    "AC": ["BASELINE"],
    "author": ["BASELINE"],
    "global_settings": {
        "provides_numeric_rating": ['reviewer', 'ac'],
        "persons_aware_of_authors_identities": []
    }
}

print("💡 커스텀 설정 예시:")
print(f"  전문가만: {expert_only_setting['reviewer']}")
print(f"  다양한 혼합: {diverse_setting['reviewer']}")
print(f"  나만의 조합: {my_custom_setting['reviewer']}")
print("\n🔧 Cell 7에서 selected_setting을 my_custom_setting으로 바꿔서 실행해보세요!")

---
## 토론

### 질문

1. **리뷰어 페르소나 조합의 영향**
   - `malicious` vs `malicious_unknowledgeable` - 차이가 있나요?
   - 3명 중 1명이 `knowledgeable_responsible`이면 품질이 올라가나요?
   - `authors_are_famous` vs `authors_are_unfamous` - 유명도 편향이 얼마나 큰가요?

2. **5단계 파이프라인의 가치**
   - Author rebuttal이 점수를 올리나요?
   - AC 토론이 편향을 줄이나요?
   - `irresponsible` 리뷰어도 rebuttal 후 점수를 바꾸나요?

3. **실제 활용**
   - Peer review 시스템 개선에 어떻게 사용?
   - 논문 투고 전 사전 테스트? (어떤 조합으로?)
   - 리뷰어 배정 최적화? (최소 몇 명이 `knowledgeable`이어야 할까?)

4. **실험 아이디어**
   - `worst_case_setting` vs `expert_only_setting` 비교
   - 같은 논문에 다양한 설정 적용하여 결과 비교
   - 어떤 조합이 실제 ICLR 리뷰와 가장 유사할까?